In [1]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from lib.main_ui import Ui_MainWindow
from lib.svc_ui import Ui_Form as SVC_Form
from lib.rf_ui import Ui_Form as RF_Form
import pandas as pd
import numpy as np
from os.path import *
from re import *
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

D:\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
main = None
top = None

In [3]:
class LearnThread(QThread):
    
    fin_sig = pyqtSignal()
    progress_sig = pyqtSignal(int)
    
    def __init__(self, parent, model):
        QThread.__init__(self, parent)
        self.model = model
    
    def run(self):
        self.model.run()
        self.fin_sig.emit()

In [4]:
# Define machine learning models
class RunModel:
    
    def __init__(self, data_file, sparm={}):
        self.data_file = data_file
        self.sparm = sparm
        
    def load_data(self):
        df = pd.read_csv(self.data_file,index_col=None)
        data = df.values
        X, Y = data[:,0:-1], data[:, -1]
        return X, Y
        
    def run(self):
        self.load_data()
        print "start"
        

class SVCModel(RunModel):
    
    def __init__(self, data_file, sparm={}):
        RunModel.__init__(self, data_file, sparm=sparm)
        
    def run(self):
        X, Y = self.load_data()
        s = SVC()
        gd = GridSearchCV(estimator=s, param_grid=self.sparm)
        gd.fit(X, Y)
        YP = gd.predict(X)
        print(classification_report(Y, YP))
        

class RFModel(RunModel):
    
    def __init__(self, data_file, sparm={}):
        RunModel.__init__(self, data_file, sparm=sparm)
        
    def run(self):
        X, Y = self.load_data()
        s = RandomForestClassifier()
        gd = GridSearchCV(estimator=s, param_grid=self.sparm)
        gd.fit(X, Y)
        YP = gd.predict(X)
        print(classification_report(Y, YP))

        
class NBModel(RunModel):
    
    def __init__(self, data_file, sparm={}):
        RunModel.__init__(self, data_file, sparm=sparm)


In [5]:
def algo_chg(idx):
    wcnt = main.algo_layout.count()
    print wcnt
    for i in range(0, wcnt):
        w = main.algo_layout.takeAt(i).widget()
        w.setParent(None)
    if idx == 1:
        w = QWidget()
        f = SVC_Form()
        f.setupUi(w)
        w.ui = f
        main.algo_layout.addWidget(w)
        w.show()
    elif idx == 2:
        w = QWidget()
        f = RF_Form()
        f.setupUi(w)
        w.ui = f
        main.algo_layout.addWidget(w)
        w.show()
        
def svc_parm(ui):
    p = {}
    p['kernel'] = [str(ui.comboBox.currentText())]
    p['decision_function_shape'] = [str(ui.comboBox_2.currentText())]
    if p['decision_function_shape'] == "default":
        p['decision_function_shape'] = [None]
    return p

def rf_parm(ui):
    p = {}
    return p

def run_sk():
    print "run"
    mtype = str(main.comboBox.currentText())
    if mtype == "":
        print "No model select"
        return
    data_file = str(main.lineEdit.text())
    if not isfile(data_file):
        print "Invalid file"
        return
    w = main.algo_layout.itemAt(0).widget()
    t = None
    if mtype == "svc":
        print "do svc"
        parm = svc_parm(w.ui)
        print parm
        t = LearnThread(top, SVCModel(data_file, parm))
    elif mtype == "random_forest":
        print "do random forest"
        parm = rf_parm(w.ui)
        print parm
        t = LearnThread(top, RFModel(data_file, parm))
    if t is None:
        return
    t.start()

def open_data():
    f = QFileDialog.getOpenFileName(top)[0]
    if f == "":
        return
    f = str(f)
    main.lineEdit.setText(f)

In [6]:
if __name__ == '__main__':
    app = QApplication(sys.argv)
    w = QMainWindow()
    top = w
    main = Ui_MainWindow()
    main.setupUi(w)
    main.algo_layout = QVBoxLayout(main.frame)
    main.comboBox.currentIndexChanged.connect(algo_chg)
    main.btn_run.clicked.connect(run_sk)
    main.pushButton.clicked.connect(open_data)
    w.show()
    sys.exit(app.exec_())

0
run
do svc
{'kernel': ['linear'], 'decision_function_shape': ['default']}
             precision    recall  f1-score   support

        1.0       1.00      0.75      0.86         4
        2.0       1.00      1.00      1.00         3
        3.0       0.67      1.00      0.80         2

avg / total       0.93      0.89      0.89         9



D:\Anaconda2\lib\site-packages\sklearn\cross_validation.py:552: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)
D:\Anaconda2\lib\site-packages\sklearn\externals\joblib\parallel.py:540: UserWarning: Multiprocessing backed parallel loops cannot be nested below threads, setting n_jobs=1
  **self._backend_args)


SystemExit: 0

D:\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
